In [1]:
##### Importation du fichier de données et de la liste des variables sélectionnées
import os
os.chdir('/Users/learegazzetti/Documents/Cours/M1_Info/Data_Mining_M1_Info/Projet')
name_data = 'data_sans_etiquettes.txt'

import pandas
Newdf = pandas.read_csv(name_data, sep = '\t')
Newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4940210 entries, 0 to 4940209
Columns: 199 entries, V1 to V199
dtypes: float64(15), int64(181), object(3)
memory usage: 7.3+ GB


In [2]:
liste_var=pandas.read_csv('variables_importantes_scoring.txt', header=None)
liste_var=liste_var[0].values
#print(liste_var)

In [3]:
#Recodage des variables qualitatives : V160,V161,V162 en variables codées en 0/1
import numpy
liste_quali=[var for var in Newdf.columns if Newdf[var].dtype==numpy.object_]

#Recodage des variables qualitatives
df_Recodage= pandas.get_dummies(Newdf[liste_quali])

#Constitution du data frame entier avec les variables quantitatives 
#et qualitatives

#Liste des variables quantitatives
liste_quanti=[var for var in Newdf.columns if Newdf[var].dtype!=numpy.object_]

#Réunir les quantitatives et les variables qualitatives recodées
data_New=pandas.concat([Newdf[liste_quanti],df_Recodage],axis=1)

In [4]:
#Sélection des variables dans le fichier de données
temp = data_New[liste_var]
data_selection = temp.values
print(data_selection.shape)

(4940210, 69)


In [5]:
#librairie chargement du modèle
import pickle

#ouverture en lecture binaire
f = open("modele_scoring.sav","rb")

#et chargement
mdl = pickle.load(f)

#fermeture du fichier
f.close()

In [6]:
#type de l'objet chargé
print(type(mdl))
print(mdl)

<class 'sklearn.linear_model._logistic.LogisticRegression'>
LogisticRegression(multi_class='ovr', solver='liblinear')


In [7]:
#calcul des probas d'affectation
probas = mdl.predict_proba(data_selection)
print(probas)

[[1.00000000e+000 3.99928567e-115]
 [1.00000000e+000 7.84019730e-020]
 [1.00000000e+000 1.81522734e-035]
 ...
 [1.00000000e+000 5.12084716e-038]
 [1.00000000e+000 4.47758446e-035]
 [1.00000000e+000 7.15649716e-046]]


In [8]:
#score de 'presence'
score = probas[:,1] 
print(score)

[3.99928567e-115 7.84019730e-020 1.81522734e-035 ... 5.12084716e-038
 4.47758446e-035 7.15649716e-046]


In [9]:
#transformation en data frame
dfScore = pandas.DataFrame(score,columns=['score'])
dfScore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4940210 entries, 0 to 4940209
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   score   float64
dtypes: float64(1)
memory usage: 37.7 MB


In [10]:
#exportation
dfScore.to_csv("scores.txt",index=False)